<a href="https://colab.research.google.com/github/sergiolucero/pydatachile/blob/master/PyData3_MercadoPublico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyData #3: Captura y Análisis de precios de insumos COVID19 con la API de ChileCompra.

El primer paso es obtener una llave (TOKEN) en https://api.mercadopublico.cl/. Aquí en adelante asumimos que esta llave, que es personal, 
se encuentra almacenada en una variable de entorno llamada ID_MERCADO:

In [0]:
import os, requests
import pandas as pd
TOKEN = os.getenv('ID_MERCADO')
# TOKEN = requests.get(token_url)
BASE = 'http://api.mercadopublico.cl/servicios/v1/publico/'
fecha = '01042020'    # formato DDMMYYYY... ¿por qué diablos?
url = f'{BASE}/ordenesdecompra.json?fecha={fecha}&estado=aceptada&ticket={TOKEN}'   # 
#url = '%s/ordenesdecompra.json?fecha=%s&estado=aceptada&ticket=%s' %(BASE,fecha,TOKEN)  # %-formatting
print(url)

http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=01042020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9


In [0]:
resp = requests.get(url).json()
print(len(resp))
resp.keys()

4


dict_keys(['Cantidad', 'FechaCreacion', 'Version', 'Listado'])

In [0]:
print('Cantidad:', resp['Cantidad'], 'Fecha:', resp['FechaCreacion'], 
      'Version:', resp['Version'])

Cantidad: 4387 Fecha: 2020-04-22T17:41:07.7 Version: v1


In [27]:
ocs = resp['Listado']
print(fecha,'tiene',len(ocs),'OCs')
ocs[3300]

30032020 tiene 4387 OCs


{'Codigo': '3950-321-SE20',
 'CodigoEstado': 6,
 'Nombre': 'ADQUISICIÓN DE FÁRMACO SEGÚN CONV. SUMINISTRO SSA.'}

In [28]:
# Usemos pandas para formatear de inmediato esta data
pd.set_option('max_colwidth', None)    # por defecto, 50 caracteres
ordenes_dia = pd.DataFrame(ocs)
ordenes_dia.Nombre = ordenes_dia.Nombre.apply(lambda n: n.lower())
ordenes_dia.tail()

,Codigo,Nombre,CodigoEstado
4382,905-76-CM20,santa cruz sc 14 /2020 no pac mantencion camioneta jfsl44,6
4383,938614-299-SE20,vme-epr servicio de aseo y mantencion de jardines,6
4384,948354-166-SE20,serv. retiro residuos especiales y peligrosos del hospital dipreca,6
4385,958433-11-SE20,anodos mtu,6
4386,999-28-SE20,servicio de aseo dpv concepción,6


Acá sólo tenemos un Codigo que corresponde al identificador de la orden de compra, y un descriptor de la compra. 
Necesitamos ocupar la otra API para obtener el detalle profundo. Como vamos a ocupar a menudo estas lógicas, las metemos en funciones de Python.

In [0]:
WAIT = 2  # segundos de respiro

def get_OC(oc):  # info detallada de una orden de compra
    BASE = 'http://api.mercadopublico.cl/servicios/v1/publico/'
    url = f'{BASE}ordenesdecompra.json?codigo={oc}&ticket={TOKEN}'
    js = requests.get(url).json()['Listado']
    return js

def get_detalle(df):  # info detallada de una lista de ordenes de compra
    pdf = pd.DataFrame()
    for oc in df.Codigo:
        print(oc, end=',')
        c0 = pd.DataFrame(get_OC(oc))
        pdf = pdf.append(c0)
        time.sleep(WAIT)   # importante darle un respiro a la API
    return pdf

def get_producto(producto, fecha):       # descargamos sólo ciertos productos
    BASE = 'http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json'
    url = '%s?fecha=%s&estado=aceptada&ticket=%s' %(BASE,fecha,TOKEN)  # f-string
    print(url)
    js = requests.get(url).json()['Listado']   # todas las OCs del día
    js = [j for j in js if producto in j['Nombre']]  # list comprehension!!!
    print(f'{len(js)} órdenes de compra de <{producto}> el dia {fecha}')

    return pd.DataFrame(js)
    

Las órdenes de compra (sin detalle) están en la columna Codigo de la variable ordenes_dia.

In [31]:
fdf = get_producto('alcohol gel','21042020')   # cuantas BOLSAs, mortuarias u de otro tipo, se compraron ese día
fdf.head(10)

http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=21042020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
24 órdenes de compra de <alcohol gel> el dia 21042020


,Codigo,Nombre,CodigoEstado
0,1002584-143-AG20,Compra de Alcohol Gel y Dispensadores,6
1,1051765-623-CM20,"ADQUISICIÓN ALCOHOL GEL, ÁREA SALUD",6
2,1718-423-AG20,M.6916 200 LITROS ALCOHOL GEL,6
3,2207-391-SE20,ADQUISICIÓN ALCOHOL GEL POR CONTINGENCIA,6
4,1979-1649-SE20,TD URGENCIA ALCOHOL GEL,6
5,2065-245-SE20,"TD "" ALCOHOL GEL DE 340 ML """,6
6,2085-353-CM20,COMPRA ( ALCOHOL GEL ),6
7,1994-69-SE20,mascarillas y alcohol gel - gobernacion,6
8,2483-260-SE20,"ADQUISICIÓN DE ALCOHOL GEL, CLORO Y JABÓN LÍQUIDO",6
9,2363-39-SE20,ALCOHOL GEL,6


Finalmente, recopilamos la información de todo un mes. Primero sería bueno estimar el tamaño de lo que vamos a ejecutar y quizás calentar el agua del café.

In [32]:
camillas = pd.DataFrame()

for dia in range(1,32):
  fecha = '%02d032020' %dia
  try:    # sugiero try except 
    fdf = get_producto('camilla',fecha)   # la data del dia
  except:
    pass
  print(fecha, len(fdf), end=':')
  fdf['fecha'] = fecha
  camillas = camillas.append(fdf)


http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=03032020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
1 órdenes de compra de <camilla> el dia 03032020
03032020 1:http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=04032020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
0 órdenes de compra de <camilla> el dia 04032020
04032020 0:http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=05032020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
1 órdenes de compra de <camilla> el dia 05032020
05032020 1:http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=06032020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
2 órdenes de compra de <camilla> el dia 06032020
06032020 2:http://api.mercadopublico.cl/servicios/v1/publico//ordenesdecompra.json?fecha=07032020&estado=aceptada&ticket=FA1D5A8D-A2CD-4204-9697-77EF124084A9
0 órdenes de compra de <c

18032020    6
17032020    6
26032020    4
20032020    3
23032020    3
27032020    3
06032020    2
25032020    2
19032020    2
12032020    1
Name: fecha, dtype: int64

Analicemos el precio de camillas por segmento.

In [42]:
def segmento_camillas(nom):
  if 'camilla rescate' in nom.lower():    return 'rescate'
  elif 'ginecologica' in nom.lower():    return 'ginecologica'
  elif 'transporte' in nom.lower():    return 'transporte'
  elif 'inmovilizadora' in nom.lower():    return 'inmovilizadora'
  else:
    return 'N/A'

camillas['segmento'] = camillas.Nombre.apply(segmento_camillas)
camillas.segmento.value_counts(normalize=True).head(5)

N/A               0.820513
transporte        0.076923
ginecologica      0.051282
rescate           0.025641
inmovilizadora    0.025641
Name: segmento, dtype: float64

In [0]:
camillas.groupby('segmento').agg()

In [33]:
camillas.fecha.value_counts().head(10)

18032020    6
17032020    6
26032020    4
20032020    3
23032020    3
27032020    3
06032020    2
25032020    2
19032020    2
12032020    1
Name: fecha, dtype: int64

In [34]:
#"POLEMICA"
OC = '1082417-18-CM20'
js = requests.get('http://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo={OC}&ticket={TOKEN}').json()
js.keys()

dict_keys(['Message'])

In [0]:
pd.DataFrame(js['Listado']).T

,0
Codigo,1082417-18-CM20
Nombre,CORONAVIRUS. Adquisición de equipos médicos por alerta sanitaria
CodigoEstado,6
Estado,Aceptada
CodigoLicitacion,
Descripcion,"CORONAVIRUS. Adquisición de equipos médicos por alerta sanitaria. Catre clínico, incluye mesa de comer y velador"
CodigoTipo,9
Tipo,CM
TipoMoneda,CLP
CodigoEstadoProveedor,4
